In [5]:
from network import SimpleCNN, SmallRegularizedCNN
from utils import CNNDataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim

import os
from datetime import datetime
import numpy as np
from network import SimpleCNN

In [2]:
dataset = CNNDataset("data/DJI_0012", input_transform = transforms.ToTensor())

In [3]:
np.sum(dataset.all_labels) / dataset.all_labels.shape[0]

0.17291520672740013

In [4]:
dataset = CNNDataset("data/DJI_0012", input_transform = transforms.ToTensor())
trainloader = DataLoader(dataset, batch_size=256, shuffle=True)



cnn = SimpleCNN()
optimizer = optim.AdamW(cnn.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[30, 50, 80], gamma=0.1)
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(5):
    running_loss = 0.0
    for data in trainloader:
        img_feature, non_spatial_feature, labels = data
        cnn.forward(img_feature, non_spatial_feature)
        #inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        preds = cnn(img_feature, non_spatial_feature)
        labels = labels.unsqueeze(1)
        loss = loss_fn(preds, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() / len(trainloader)

    scheduler.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, running_loss ))

print('Finished Training')
    

KeyboardInterrupt: 

In [6]:
model = SmallRegularizedCNN()

In [9]:
model_state = torch.load('models/checkpoint.pt')
model.load_state_dict(model_state)

<All keys matched successfully>

In [10]:
torch.save(model.state_dict(), 'models/SmallRegularizedIntent-4-10-22.pt')